# Putting it Together

In the previous section we introduced blocks, the fundamental building blocks of a pipeline in Bifrost.  Now we will talk about how blocks are connected together and some of the considerations.

In Bifrost blocks are connected together by circular memory buffers called "rings".  Like a `bifrost.ndarray`, a ring exists in a memoy space:  system, cuda_host, or cuda.  A ring also has a size that based on a integer number of segments of the gulp size for the ring. 

To create a ring in Bifrost:

In [1]:
import bifrost

ring = bifrost.ring.Ring(name="a_ring", space="system")
print('name:', ring.name, ', space:', ring.space)

name: b'a_ring' , space: system


This creates a new ring, called "a_ring", in the system memory space.  Although the ring has been created it does not yet have any memory allocated to it.  To allocate memory you `resize` it:

In [2]:
ring.resize(4096)

This sets the gulp size for the ring to 4096 bytes and this call sets the total ring size to four, 4096 byte buffer.  You can change the buffer fraction by adding in a second argument which is the total ring size.  For example, to increase the buffer size to five segments:

In [3]:
ring.resize(4096, 5*4096)

Resizing a ring is a data-safe process and the contents of the ring are preserved.

Rings in Bifrost are more than just a section of memory, though.  It has a few other attributes that make it useful for representing a stream of data:

 * a timetag that denotes when the stream of data starts
 * a header that stores metadata about the sequence
 * they support single writer/multi-reader access for branching pipelines

Let's use an example to look at these first two.  In this we will write some data to the ring:

In [4]:
import json, numpy, time

ring = bifrost.ring.Ring(name="another_ring", space="system")

with ring.begin_writing() as output_ring:
    time_tag = int(time.time()*1e9)
    hdr = {'time_tag':      time_tag,
           'metadata':      'here',
           'more_metadata': 'there'}
    hdr_str = json.dumps(hdr)
    
    gulp_size = 4096
    ring.resize(gulp_size, 5*gulp_size)
    
    with output_ring.begin_sequence(time_tag=hdr['time_tag'],
                                    header=hdr_str) as output_seq:
        for i in range(20):
            with output_seq.reserve(gulp_size) as output_span:
                data = output_span.data_view(numpy.int8)
                data[...] = (numpy.random.rand(gulp_size)\
                             *127).astype(numpy.int8)
                print(i, '@', data[:5])

0 @ [[ 38 117  68 ...  75  66  96]]
1 @ [[ 78  28  37 ...  70 121  75]]
2 @ [[ 88  41 106 ...  90  69 103]]
3 @ [[ 98  18  64 ...  34  65 101]]
4 @ [[ 90 115  10 ...  41  94 118]]
5 @ [[ 16   3 119 ...  28  88  31]]
6 @ [[ 32  78  32 ... 126  17  64]]
7 @ [[ 13   7   6 ... 109 103 116]]
8 @ [[ 50  12  67 ... 126 123   8]]
9 @ [[ 79   8 118 ... 114  97  95]]
10 @ [[87 28 74 ...  2 74 63]]
11 @ [[  1  94 123 ...  41  14 112]]
12 @ [[  0  45  96 ...  51 102  49]]
13 @ [[54 88 56 ... 74 20 30]]
14 @ [[ 23  98 101 ... 126  51  21]]
15 @ [[ 49   1 101 ... 100  40  36]]
16 @ [[97 25 50 ... 90 83 38]]
17 @ [[ 90 105  85 ...  66  81  19]]
18 @ [[  5  80  81 ...  11  16 111]]
19 @ [[ 65 125   4 ...  25  97  11]]


Here we:

 1. Ready the ring for writing with `ring.begin_writing()`.
 2. Once the ring is ready for writing, we define the time tag for the first sample and a dictionary of metadata.  The time tag is expected to be an integer and the dictionary is dumped to a JSON object.
 3. Start a "sequence" on the ring using that time tag and JSON object. 
  * In Bifrost a sequence is a stream of data with a single observational setup.
 4. Loop over spans, also called gulps, in the output sequence and writes data to the ring.
  * Writing uses a "data_view" of the span/gulp that exposes it as a `bifrost.ndarray`.

Reading from a ring follows a similar sequence:

In [5]:
for input_seq in ring.read(guarantee=True):
    hdr = json.loads(input_seq.header.tobytes())
    print(input_seq.time_tag)
    print(hdr)
    
    gulp_size = 4096
    
    i = -1
    for input_span in input_seq.read(gulp_size):
        i += 1
        if input_span.size < gulp_size:
            continue
        data = input_span.data_view(numpy.int8)
        print(i, '@', data[:10])

1619206567011656448
{'time_tag': 1619206567011656448, 'metadata': 'here', 'more_metadata': 'there'}
12 @ [[  0  45  96 ...  51 102  49]]
13 @ [[54 88 56 ... 74 20 30]]
14 @ [[ 23  98 101 ... 126  51  21]]
15 @ [[ 49   1 101 ... 100  40  36]]
16 @ [[97 25 50 ... 90 83 38]]
17 @ [[ 90 105  85 ...  66  81  19]]
18 @ [[  5  80  81 ...  11  16 111]]
19 @ [[ 65 125   4 ...  25  97  11]]


/home/jdowell/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: generator 'ReadSequence.read' raised StopIteration
  if __name__ == '__main__':
/home/jdowell/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: generator 'Ring.read' raised StopIteration
  """Entry point for launching an IPython kernel.


Here we:

 1. Open the ring for reading with `ring.read()` and get an iterator over sequences in that ring.
  * This ring was opened with `gaurantee=True` which tells Bifrost that spans that are being read from cannot be overwriten with new data until the reader releases the span.
 2. For the sequence we can access its time_tag and metadata header.
 3. Loop over spans/gulps within that sequence until the iterator is exhausted.
  * It is possible that a span returned by `input_seq.read()` is smaller than the gulp size, particuarlly at the end of a sequence.  It is a good idea to check the size of the span before trying to use it.
 4. For each span, do the processing that is required.

In the next section we will talk about how to build a complete pipeline from these pieces. 